In [1]:
import pandas as pd 
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode
import ast

In [2]:
path = r'.\scrapping_data.csv'

df = pd.read_csv(path)
df

,title,abstract,date_publication,keywords
0,From Misaligned Sub-Saturns to Aligned Brown D...,We present a pattern emerging from stellar obl...,12/5/2024,"['exoplanet dynamics (490)', ' star-planet int..."
1,Uniting the Observed Dynamical Dark Energy Pre...,"Recent results from Type Ia Supernovae (SNe), ...",12/5/2024,"['supernovae', 'physics', 'lambdaroman_λcdm', ..."
2,Journey to the center of the common envelope e...,Three-dimensional hydrodynamical simulations o...,12/5/2024,['Key Words.: Hydrodynamics – binaries: close ...
3,Baryon Acoustic Oscillations from galaxy surve...,We conducted a review of the fundamental aspec...,12/5/2024,"['galaxies', 'alphaitalic_α', 'spectroscopic',..."
4,Optical constraints on the coldest metal-poor ...,Context. The coldest metal-poor population mad...,12/5/2024,"['coldest', 'old', 'dwarfs', 'archaeological',..."
...,...,...,...,...
1124,A sharp bound for the functional calculus of $...,Abstract: Let $A$ be a $\rho$-contraction and ...,12/3/2024,"['function', 'frac', 'contraction', 'disk', 'm..."
1125,An Archimedean Vector Lattice Functional Calcu...,Abstract: We develop a functional calculus on ...,12/3/2024,"['calculus', 'formulas', 'bounded', 'archimede..."
1126,Wigner analysis of operators. Part III: Contro...,Abstract: The integration of operator kernels ...,12/2/2024,"['1932', 'symplecticmappings', 'calculus', 'ou..."
1127,On the numerical radius parallelism and the nu...,"Abstract: In this paper, we generalize the not...",12/2/2024,"['birkhofforthogonality', 'numericalradius', '..."


In [32]:
def count_keywords(row):
    try:
        keyword_str = row.split(']')[0] + ']'
        keywords = ast.literal_eval(keyword_str)
        return len(keywords)
    except Exception as e:
        return 0
    

In [39]:
def split_keywords(row):
    try:
        keyword_str = row['keywords'].split(']')[0] + ']'
        keywords = ast.literal_eval(keyword_str)
        keywords_clean = [kw.split('(')[0].strip() for kw in keywords]  # Clean up keywords
        return [keyword for keyword in keywords_clean]
    except Exception as e:
        return [] 

In [40]:
df['keyword_count'] = df['keywords'].apply(count_keywords)

df = df.explode('keywords').reset_index(drop=True)
df_keywords = df['keywords'].str.extract(r'^(.+?) \(')

In [ ]:
rows = []
for _, row in df.iterrows():
    rows.extend(split_keywords(row))

new_df = pd.DataFrame(rows, columns=['Keyword'])

new_df

In [ ]:
new_df.to_csv()